### installing vx-underground & azure-storage

In [ ]:
#!pip install mwdblib
#!pip install azure-storage-blob

### importing library

In [ ]:
import boto3, json, warnings, datetime

from datetime import date, datetime, timedelta

warnings.filterwarnings("ignore")

### importing secrets [azure & vx] from aws secrets manager - replace "xxxxxxxxxxxxxxxxxx" with secret arn

In [ ]:
secrets_client = boto3.client('secretsmanager')

secret_arn = 'xxxxxxxxxxxxxxxxxx'
azure_auth_token = secrets_client.get_secret_value(SecretId=secret_arn).get('SecretString')
azure_auth_token = json.loads(azure_auth_token)

secret_arn = 'xxxxxxxxxxxxxxxxxx'
vx_auth_token = secrets_client.get_secret_value(SecretId=secret_arn).get('SecretString')
vx_auth_token = json.loads(vx_auth_token)

### azure_storage_key, vx_key are secret name

In [ ]:
azure_key = azure_auth_token.get("azure_storage_key")
vx_key = vx_auth_token.get("vx_key")

### generating blob searvice client, replace "xxxxxxxxxxxxxxxxxx" with azure account name & container name

In [ ]:
from azure.storage.blob import BlobServiceClient, BlobSasPermissions, generate_blob_sas
service = BlobServiceClient(account_url="https://xxxxxxxxxxxxxxxxxx.blob.core.windows.net/", credential=azure_key)

### getting sha1 [latest 100K] data from vx-underground and saving it in "VX-Underground-SHA1-DB.csv" in current directory

In [ ]:
from mwdblib import MWDB
from itertools import islice

file_name = 'VX-Underground-SHA1-DB.csv'

print("\nDownloading SHA1 Data to Local Syatem as CSV:\t" + file_name)

mwdb = MWDB(api_url='https://virus.exchange/api',api_key=vx_key)

files = islice(mwdb.recent_files(), 99999)
with open(file_name, 'w') as intel:
    for f in files:
        intel.write(f.sha1)
        intel.write(',\n')

### uploading "VX-Underground-SHA1-DB.csv" to azure storage - replace "xxxxxxxxxxxxxxxxxx" with container name 

In [ ]:
import os, uuid
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, __version__

file_name = 'VX-Underground-SHA1-DB.csv'

container_client = service.get_container_client(container="xxxxxxxxxxxxxxxxxx")

print("\nUploading to Azure Storage as CSV File:\t" + file_name)

with open(file_name, "rb") as data:
    container_client.upload_blob(name=file_name, data=data,blob_type="AppendBlob",overwrite=True)

print("\n\nAll Done")

### generating sas token to import data in mde kql

In [ ]:
account_name = "xxxxxxxxxxxxxxxxxx"
container_name = "xxxxxxxxxxxxxxxxxx"

def generate_sas_token(file_name):
    
    sas = generate_blob_sas(account_name=account_name,
                            account_key=azure_key,
                            container_name=container_name,
                            blob_name=file_name,
                            permission=BlobSasPermissions(read=True),
                            expiry=datetime.utcnow() + timedelta(hours=2200))

    sas_url ='https://'+account_name+'.blob.core.windows.net/'+container_name+'/'+file_name+'?'+sas
    return sas_url

sas_token = generate_sas_token(file_name)

print(sas_token)

### kql example - replace "xxxxxxxxxxxxxxxxxx" with sas token
    - either run this query manually in Advanced Hunting or create a custome alert for scheduled run

In [ ]:
let vx_underground = materialize(externaldata(SHA1:string)
    [@"xxxxxxxxxxxxxxxxxx"]
    with (format="csv", ignoreFirstRecord=false)
);
union (vx_underground | join (AlertEvidence | where isnotempty(SHA1)) on SHA1),(vx_underground | join (DeviceEvents | where isnotempty(SHA1)) on SHA1),
(vx_underground | join (DeviceImageLoadEvents | where isnotempty(SHA1)) on SHA1),(vx_underground | join (DeviceFileCertificateInfo | where isnotempty(SHA1)) on SHA1),
(vx_underground | join (DeviceFileEvents | where isnotempty(SHA1)) on SHA1),(vx_underground | join (DeviceProcessEvents | where isnotempty(SHA1)) on SHA1)
| summarize count() by SHA11
| sort by count_

#### you can use cron job to schedule this task [downloading vx-underground latest data and uploading it to azure storage]